In [38]:
import os
import pandas as pd
from pymystem3 import Mystem
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.pipeline import  make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [43]:
csv_file = 'with_lemma.csv' #'labeled.csv'
RANDOM_STATE = 42

m_steam = Mystem()

nltk.download('stopwords')

stopwords = list(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mukha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
data = pd.read_csv(csv_file)

In [48]:
#data.head()

In [49]:
#data.info()

In [28]:
#Отображает гистаграму распределения классов
def histogram_show(data, col):
    
    data[col].hist()
    plt.title('Количество комментариев')
    plt.xlabel('Значение')
    plt.ylabel('Количество')
    plt.show()
    

In [46]:
#histogram_show(data,'toxic')

In [30]:
#отоброжает процентное соотношение классов
def precent_show(data):
    data.toxic.value_counts(normalize=True).plot(kind='bar')
    plt.title('% toxic')
    plt.ylabel('%')
    plt.xlabel('Классы')
    plt.show()


In [45]:
#precent_show(data)

In [36]:
# Функция для лемматизация текста и очистки 
def lemma_text(text):
    lemaized_text = " ".join(m_steam.lemmatize(text))
    cleare_text =  re.sub(r'[^а-яА-ЯёЁ ]', ' ',lemaized_text)
    print(cleare_text)
    return " ".join(cleare_text.split())

In [35]:
#data['lemma_text'] = data['comment'].apply(lemma_text)

In [55]:
data = data.dropna()
data['lemma_text'].drop_duplicates(keep='first' , inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14404 entries, 0 to 14411
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   comment     14404 non-null  object 
 1   toxic       14404 non-null  float64
 2   lemma_text  14404 non-null  object 
dtypes: float64(1), object(2)
memory usage: 450.1+ KB


Разделим данные на бучающию и тестовую выборку

In [52]:
#Разделим данные
train_x, test_x , train_y, test_y = train_test_split(data['lemma_text'], data['toxic'], test_size=0.2)


In [59]:


#Создадим и обучим модель LogisticRegression с балансировкой весов
pipeline = make_pipeline(count_tf_idf,
                              LogisticRegression(penalty='l1', solver='saga',class_weight='balanced',max_iter=5000))

params={
    'logisticregression__C':[4, 10, 2]
}

gs_lr = GridSearchCV(pipeline,
                      params,
                      cv=2,
                      scoring='accuracy')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mukha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:


gs_lr.fit(train_x, train_y)
print('F1 score', gs_lr.best_score_)
print('Best model parameters', gs_lr.best_estimator_)


F1 score 0.8410136237266939
Best model parameters Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['и', 'в', 'во', 'не', 'что', 'он',
                                             'на', 'я', 'с', 'со', 'как', 'а',
                                             'то', 'все', 'она', 'так', 'его',
                                             'но', 'да', 'ты', 'к', 'у', 'же',
                                             'вы', 'за', 'бы', 'по', 'только',
                                             'ее', 'мне', ...])),
                ('logisticregression',
                 LogisticRegression(C=4, class_weight='balanced', max_iter=5000,
                                    penalty='l1', solver='saga'))])
